# Stay Subway Safe

## Front End - Interface
To interact with our wireframe (made with JustInMind), click here: https://www.justinmind.com/usernote/tests/47687661/47738106/47746104/index.html

## Backend, with logic explanations

### User input
These variable are defined by user input, as depicted in wireframe.

For this demonstration, values are hard coded and we assume the user wants to know what is happening NOW.

In [1]:
from datetime import datetime
import numpy as np

In [2]:
station = "96 ST"
trainline = "1"
date = datetime.date(datetime.now())
time = datetime.time(datetime.now())

### Gather necessary data to fulfill request

In [3]:
import os
import sys
sys.path.insert(0, os.path.abspath('/Users/mehrkaur/Documents/projects/subway/MassTransit'))

In [4]:
from get_station_csv import get_final_station_data
from sort_by_week_day import sort_all_days
from h4_chunks_in_py import entries_exits_in_4h_chunks

We scrape the most recent week's data for this requested station...

In [5]:
get_final_station_data(1, station, trainline)

...then sort that data by weekday and create individual csv files.

In [6]:
sort_all_days(station)

If the user's requested day is a work day, the prediction will be based on average crowd size across all work days from the previous week during the requested time window. This mitigates the impact of anomolies from the previous week on the prediction. 

If the user requests info for a Saturday or Sunday, we look specifically at the previous Saturday or Sunday. This is because the MTA runs fewer trains on Sundays than other days of the week.

In [7]:
requestedDay = date.weekday()
weekDays = ("Mon","Tue","Wed","Thu","Fri","Sat","Sun")

We use our other modules to get the number of train arrivals and number of turnstile entries per hour.

Train arrivals are specific to the day of the week but not the date. we will get this value now so that we don't have to call it multiple times in the loop below. However, turnstile entries are date specific, so we will call when we need it.

In [8]:
from train_info_from_schedule import single_frequency
from entries_exits import get_entry_exit_given_hour

In [10]:
stops_path = '/Users/mehrkaur/Documents/projects/subway/MassTransit/google_transit/stop_times.txt'
trips_path = '/Users/mehrkaur/Documents/projects/subway/MassTransit/google_transit/trips.txt'
stop_id = "120" #HARDCODED for 96th, our pilot

#train schedule is recorded only between 6 am (06) and 11 pm (23)
#also, time needs to be formatted to have 2 digits
hourINT = time.hour
if (hourINT < 6):
    hour = "06"
elif (hourINT > 23):
    hour = "23"
elif (hourINT < 10):
    hour = "0" + str(hourINT)
else:
    hour = str(hourINT)

#train schedule data is split into weekday, Saturday, Sunday
if(requestedDay <= 4):
    day_type = "Weekday"
elif(requestedDay == 5):
    day_type = "Saturday"
else:
    day_type = "Sunday"



northTrains = single_frequency(stops_path,trips_path,stop_id,hour,day_type,"Northbound")
southTrains = single_frequency(stops_path,trips_path,stop_id,hour,day_type,"Southbound")

trainsPerPlatform = np.array([northTrains, southTrains])

/Users/mehrkaur/Documents/projects/subway/MassTransit/train_info_from_schedule.py:67: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  source['hour_of_day'][item]=source['arrival_time'][item][:2]


reformat time as needed to get turnstile entry information

time_input should be in 24 hr hh:mm 

In [11]:
mins = str(time.minute)
if(len(mins) == 1):
    mins = "0" + mins

time_input = hour + ":" + mins

#our turnstile data is available from 06:00 - 20:59
#we already account for 06:00 bound above, we need to now account for 20:59 bound here.
if(hourINT > 20):
    time_input = "20:59"

### The meat and potatoes: The Algorithm

1) totalEntries = get total station entries in that time chunk

2) entriesPerPlatform = totalEntries / # of platforms

3) trainsPerPlatform = a list of the number of trains arriving to each relavent platform in the time window

4) predictedCrowds = entriesPerPlatform / trainsPerPlatform 
    a list of max crowd size for each platform
    
5) return predictedCrowds to interface

This algorithm tells the user how large, on average, crowd size becomes before the train arrives.


96th street has 2 in service island platforms https://en.wikipedia.org/wiki/96th_Street_station_(IRT_Broadway%E2%80%93Seventh_Avenue_Line) 
We hardcode this value for the pilot program

In [12]:
totalEntries = 0
totalExits = 0 #throw-away value
predictedCrowds = np.array([0,0]) #HARDCODED for 96th. 2 platforms, 2 crowd predictions.

if(requestedDay <= 4): #is a work day
    sumCrowds = np.array([0,0]) #HARDCODED for 96
    for i in range (0,5):
        filename = weekDays[i] + ".csv"
        path = '/Users/mehrkaur/Documents/projects/subway/MassTransit/' + filename
        totalEntries, totalExits = get_entry_exit_given_hour(path,time_input) 
        entriesPerPlatform = totalEntries / 2 #HARDCODED, num. platforms at 96th
        
        tempCrowds = entriesPerPlatform / trainsPerPlatform
        sumCrowds = sumCrowds + tempCrowds

    predictedCrowds = sumCrowds / 5 #avg over 5 work days
         
else: #is weekend
    filename = weekDays[requestedDay] + ".csv"
    path = '/Users/mehrkaur/Documents/projects/subway/MassTransit/' + filename
    totalEntries, totalExits = get_entry_exit_given_hour(path,time_input)
    entriesPerPlatform = totalEntries / 2 #HARDCODED, num. platforms at 96th

    predictedCrowds = entriesPerPlatform / trainsPerPlatform
    

/Users/mehrkaur/Documents/projects/subway/MassTransit/entries_exits.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  item['ENTRIES']=entries
/Users/mehrkaur/Documents/projects/subway/MassTransit/entries_exits.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  item[item.columns[11]]=exits


In [13]:
print(totalEntries)
print(predictedCrowds)

213.0
[6.26470588 7.60714286]


predictedCrowds is the return value of the algorithm

### Grader
Color codes the returned crowd size depending on the size of the selected platforms.

In [14]:
#in progress - Andrew
platformDensities = np.array([0,0])
platform_length = 510
platform_width = 12.5
platform_size = platform_length * platform_width
if predictedCrowds:
    length = len(predictedCrowds)
    platformDensities = np.array(length)
    for i in length:
        if predictedCrowds[i]: density = predictedCrowds[i] / platform_size
        else: density = 0
        platformDensities[i] = density
else: platformDensities = predictedCrowds

platformGrades = np.array(["",""])
green = 0.05
yellow = 0.1
orange = 0.25
if platformDensities:
    length = len(platformDensities)
    platformGrades = np.array(length)
    for i in length:
        if platformDensities[i]:
            if (platformDensities[i] < green): platformGrades[i] = "G"
            elif (platformDensities[i] >= green):
                if (platformDensities[i] >= orange): platformGrades[i] = "R"
                elif (platformDensities[i] < orange): platformGrades[i] = "Y"
                else: platformGrades[i] = "O"
        else: platformGrades[i] = ""
else: platformGrades = np.array(["",""])

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

### Vizualization

add the heat map stuff here, including the "over the course of the day" visual raayan mentioned